# Backtest
In this section, I will backtest an investment strategy that enters Draftkings DFS contests based on my player statistic projections. I *temporarily* limit the backtest to Draftkings contests due to the current lack of data on the historical results of DFS contests on other sites. The methods that determine which contests I will enter and the lineups that enter each contest will be explored in this notebook. 

## Setup

Import the necessary libraries.

In [1]:
# for data
from pyNBA.Data.constants import ROTO_NAME_TO_NBA_NAME, DB_TEAM_TO_NBA_TEAM, BAD_CONTEST_SUBSTRINGS, OWNERSHIP_NAME_TO_NBA_NAME
from pyNBA.Data.data import QueryData
from pyNBA.Models.helpers import CleanData
import pandas as pd
import numpy as np
from numpy.random import normal
from datetime import datetime, timedelta

# for models
from pyNBA.DFS.rules import FPCalculator
from pyNBA.DFS.constants import Site
from pyNBA.Models.fantasy_points import FPModel
from pyNBA.Models.variance import VarianceModel
from pyNBA.Models.ownership import OwnershipModel
from pyNBA.Models.top_score import TopScoreModel
import pulp
from pydfs_lineup_optimizer.solvers.pulp_solver import PuLPSolver

# for backtesting
from pyNBA.Models.features import FeatureCreation
import time
from pydfs_lineup_optimizer import get_optimizer, Sport
from pydfs_lineup_optimizer import Player
from numpy.random import normal
from numpy.random import choice
import scipy.stats as st
import random
from random import sample
from collections import Counter
import bisect

# for plotting
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
from datetime import datetime
import seaborn as sns; sns.set()

# misc
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

## Data
Collect all data necessary to perform the backtest.

In [2]:
query_data = QueryData()
clean_data = CleanData()

# pull boxscore data
boxscores = query_data.query_boxscore_data()

# we are only interested in the boxscores of healthy and non-resting players
boxscores = clean_data.drop_rows_player_injured(boxscores)
boxscores = clean_data.drop_rows_player_rest(boxscores)

# historical DFS salary data
salary_data = query_data.query_salary_data()
salary_data = salary_data.rename(columns={'POSITION': 'DFS_POSITION'})
salary_data['NAME'] = salary_data['PLAYER'].apply(clean_data.roto_name_to_nba_name)

# historical DFS contest data. I am only interested in predicting ownership in Classic GPP compeititions.
contest_data = query_data.query_contest_data()
contest_data['MAXROI'] = contest_data['TOPPRIZE']/contest_data['ENTRYFEE']
contest_data = contest_data.loc[
    (contest_data['SLATETYPE'] == 'Classic') & (contest_data['CASHLINE'] > 200) & (contest_data['MAXROI'] > 2) &
    (~contest_data['CONTESTNAME'].str.lower().str.contains('|'.join(BAD_CONTEST_SUBSTRINGS)))
].dropna(subset=['CASHLINE'])

# contest payout structure
contest_info = query_data.query_contest_info_data()

# historical player ownership
ownership_data = query_data.query_ownership_data()
ownership_data['NAME'] = ownership_data['PLAYERNAME'].apply(
    lambda x: x if x not in OWNERSHIP_NAME_TO_NBA_NAME else OWNERSHIP_NAME_TO_NBA_NAME[x]
    )

# historical vegas odds data
odds_data = query_data.query_odds_data()

# historical quarterly boxscore data
quarterly_boxscore_data = query_data.query_quarterly_boxscore_data()

/Users/brandonshimiaie/Projects/pyNBA/sqlite/db/nba.db
2.6.0


## Strategy
Determine viable contests and how to enter each contest.

Initilize data.

In [3]:
season_list = list(boxscores['SEASON'].unique())

# to calculate fantasy points
DKFPCalculator = FPCalculator(Site.DRAFTKINGS)

# isolate Draftkings salaries
boxscores['REB'] = boxscores['DREB'] + boxscores['OREB']
    
# optimize Draftkings NBA lineups
class CustomPuLPSolver(PuLPSolver):
    LP_SOLVER = pulp.GLPK_CMD(msg=0)
optimizer = get_optimizer(Site.DRAFTKINGS, Sport.BASKETBALL, solver=CustomPuLPSolver)

Helper Functions.

In [4]:
def get_contest_prize(contest_info, score):
    prize = contest_info.loc[contest_info['MINPOINTS'] <= score, 'PRIZE'].max()
    return prize

def get_complete_positions(position_list):
    if bool(set(position_list).intersection(['PG', 'SG'])):
        position_list.append('G')
    if bool(set(position_list).intersection(['SF', 'PF'])):
        position_list.append('F')
    position_list.append('U')
    return position_list

def first_valid_position(l1, l2):
    for i in l1:
        if i not in l2:
            return l2 | {i}
        
def get_n_opponent_generations(max_contest_total_entries):
    if max_contest_total_entries < 10000:
        return 3.5
    elif max_contest_total_entries < 50000:
        return 2.5
    elif max_contest_total_entries < 100000:
        return 1.5
    else: 
        return 1
    
def get_prizes(my_scores, opponent_scores, n_opponent_scores, rank_to_prize, entry_fee):
    my_prizes = []

    prev_rank = 0
    for my_score in my_scores:
        if prev_rank == n_opponent_scores:
            my_prizes.append(0 - entry_fee)
        else:
            rank = prev_rank + bisect.bisect_left(opponent_scores[prev_rank:], my_score)
            prize = rank_to_prize[rank+1] if rank+1 in rank_to_prize else 0
            my_prizes.append(prize - entry_fee)
            prev_rank = rank
    
    return my_prizes

Enter contests.

In [ ]:
print(time.ctime())

max_entry_fee_per_contest = 1000
randomness_factor = 2
min_salary = 49200
salary_room = 1000
n_opponent_split = 3
max_n_samples = 5
n_simulations = 1000
timeout_minutes = 10

results = pd.DataFrame(columns = [
    'DATE', 'GAME_COUNT', 'CONTEST_TYPE', 'MAX_ENTRIES', 'TOTAL_ENTRIES',
    'ENTRY_FEE', 'CASHLINE', 'SCORE', 'TOP_SCORE', 'PRIZE', 'TOP_PRIZE'
])
pnl = 0

for i in range(1, len(season_list)):
    print(season_list[i])
    fp_train_data = boxscores.loc[boxscores['SEASON'].isin(season_list[0:i])]
    ownership_train_data = boxscores.loc[~(boxscores['SEASON'] == season_list[i])]
    test_data = boxscores.loc[boxscores['SEASON'] == season_list[i]]

    # projected contest top scores
    contest_test_data = contest_data.loc[
        (contest_data['DATE'] >= test_data['DATE'].min()) & (contest_data['DATE'] <= test_data['DATE'].max())
    ]
    contest_train_data = contest_data.loc[
        ~((contest_data['DATE'] >= test_data['DATE'].min()) & (contest_data['DATE'] <= test_data['DATE'].max()))
    ]
    if contest_test_data.empty:
        continue

    # projected fantasy points
    fp_model = FPModel(fp_train_data, test_data, Site.DRAFTKINGS)
    fp_model.train_model(quarterly_boxscore_data, odds_data)
    predictions, fp_hat_column = fp_model.predict()
    predictions = predictions.dropna(subset=[fp_hat_column])

    # actual fantasy points
    predictions['DKFP'] = predictions.apply(
        lambda x: DKFPCalculator.calculate_fantasy_points(
            x['SEASON'], x['PTS'], x['REB'], x['AST'], x['TOV'], x['BLK'], x['STL'], x['FG3M']
        ), 
        axis=1
    )

    # fantasy point variance
    variance_model = VarianceModel(predictions)
    predictions, variance_column = variance_model.predict('DKFP')
    pct_variance_column = 'RESIDUAL_PCT'
    predictions[pct_variance_column] = predictions[variance_column]/predictions[fp_hat_column]
    predictions[pct_variance_column] = predictions[pct_variance_column].replace([np.inf, -np.inf], np.nan)
    predictions[pct_variance_column] = predictions[pct_variance_column].fillna(0.35)
    predictions[pct_variance_column] = predictions[pct_variance_column].clip(lower=0.15)

    # projected ownership
    ownership_model = OwnershipModel(ownership_train_data, test_data, Site.DRAFTKINGS)
    ownership_model.create_features(salary_data, contest_data, ownership_data, odds_data)
    ownership_model.train_model()
    ownership_predictions, ownership_column = ownership_model.predict()

    # merge projections to data
    dk_salary_data = salary_data.loc[salary_data['SITE'] == Site.DRAFTKINGS]
    predictions = predictions.merge(dk_salary_data, on=['DATE', 'NAME'], how='left')
    predictions = predictions.dropna(subset=['SALARY', 'DFS_POSITION'])

    ownership_predictions = ownership_predictions.merge(
        predictions[['DATE', 'NAME', fp_hat_column, pct_variance_column, 'SALARY']],
        on=['DATE', 'NAME'], how='left'
    )

    print(time.ctime())

    # iterate through each slate in the season
    a = contest_test_data.loc[contest_test_data['TOTALENTRIES'] <= 250000].sort_values(by='DATE')
    for (date, slate_id), slate_contest_data in a.groupby(['DATE', 'SLATEID']):
        print(time.ctime(), date, slate_id)

        # get players in slate
        slate_teams = slate_contest_data.iloc[0]['TEAMS'].split('_')
        slate_teams = [i if i not in DB_TEAM_TO_NBA_TEAM else DB_TEAM_TO_NBA_TEAM[i] for i in slate_teams]
        board = predictions.loc[
            (predictions['DATE'] == date) &
            (predictions['TEAM'].isin(slate_teams))
        ]

        board_teams = list(board['TEAM'].unique())
        game_count = slate_contest_data.iloc[0]['GAMECOUNT']
        sym_dif = (set(board_teams).symmetric_difference(set(slate_teams)))
        if bool(sym_dif) or len(slate_teams) != game_count*2:
            print('ERROR: MISSING TEAM DATA')
            continue

        slate_ownership_predictions = ownership_predictions.loc[ownership_predictions['SLATEID'] == slate_id]
        slate_board = slate_ownership_predictions[['DATE', 'NAME', ownership_column]].merge(board, on=['DATE', 'NAME'], how='left')
        slate_board = slate_board.dropna(subset=['SALARY', 'DFS_POSITION', fp_hat_column, pct_variance_column, ownership_column])
        ownership_sum = slate_board[ownership_column].sum()
        slate_board[ownership_column] = slate_board[ownership_column].map(lambda x: 8*x/ownership_sum)
            
        # generate opponent lineups for slate
        max_contest_total_entries = slate_contest_data['TOTALENTRIES'].max()
        opponent_players = slate_board.loc[slate_board[ownership_column]*max_contest_total_entries >= 1]
        opponent_players['DFS_POSITIONS'] = opponent_players['DFS_POSITION'].map(lambda x: x.split('_'))
        opponent_players['DFS_POSITIONS'] = opponent_players['DFS_POSITIONS'].map(lambda x: get_complete_positions(x))

        player_counter = Counter()
        max_salary = opponent_players['SALARY'].max()
        n_opponent_generations = get_n_opponent_generations(max_contest_total_entries)
        n_rows = int(max_contest_total_entries/n_opponent_split)
        prev_n_valid_opponent_lineups = 0

        all_opponent_lineups = pd.DataFrame()
        min_lineups = n_opponent_generations*max_contest_total_entries
        end_time = datetime.now() + timedelta(minutes=timeout_minutes)
        while len(all_opponent_lineups) < min_lineups and datetime.now() < end_time:
            opponent_lineups = pd.DataFrame(
                {'PLAYERIDS': np.nan, 'SPOTS_REMAINING': 8, 'POSITIONS_FILLED': np.nan, 'SALARY_REMAINING': 50000},
                index = [i for i in range(n_rows)]
            )
            opponent_lineups['PLAYERIDS'] = [list()] * len(opponent_lineups)
            opponent_lineups['POSITIONS_FILLED'] = [set()] * len(opponent_lineups)
            n_opponent_lineups = len(opponent_lineups)
            
            if prev_n_valid_opponent_lineups == 0:
                opponent_players = opponent_players.sample(frac=1)
            else:
                opponent_players['COUNT'] = opponent_players['PLAYERID'].map(lambda x: player_counter[x])
                opponent_players['EXPECTED_COUNT'] = opponent_players[ownership_column]*len(all_opponent_lineups)
                opponent_players['REP'] = opponent_players['COUNT'] - opponent_players['EXPECTED_COUNT']
                opponent_players = opponent_players.sort_values(by='REP')
            for _, player in opponent_players.iterrows():
                valid_indexes = opponent_lineups.loc[
                    ~(opponent_lineups['POSITIONS_FILLED'].map(set(player['DFS_POSITIONS']).issubset)) &
                    (opponent_lineups['SALARY_REMAINING'] > player['SALARY']) & 
                    (opponent_lineups['SALARY_REMAINING'] - (opponent_lineups['SPOTS_REMAINING']-1)*max_salary - salary_room < player['SALARY'])
                ].index.to_list()
                n = len(valid_indexes)
                len_subset = int(min(n, (n_rows*player[ownership_column])))

                subset = sample(valid_indexes, len_subset)
                opponent_lineups.loc[subset, 'PLAYERIDS'] = opponent_lineups.loc[subset, 'PLAYERIDS'].map(lambda x: x + [player['PLAYERID']])
                opponent_lineups.loc[subset, 'SPOTS_REMAINING'] -= 1
                opponent_lineups.loc[subset, 'POSITIONS_FILLED'] = opponent_lineups.loc[subset, 'POSITIONS_FILLED'].map(
                    lambda x: first_valid_position(player['DFS_POSITIONS'], x)
                )  
                opponent_lineups.loc[subset, 'SALARY_REMAINING'] -= player['SALARY']

            valid_opponent_lineups = opponent_lineups.loc[opponent_lineups['SPOTS_REMAINING'] == 0]
            finished_lineups = valid_opponent_lineups['PLAYERIDS'].to_list()
            for lineup in finished_lineups:
                for player in lineup:
                    player_counter[player] += 1
            
            prev_n_valid_opponent_lineups = len(valid_opponent_lineups)
            all_opponent_lineups = all_opponent_lineups.append(valid_opponent_lineups)
            
        if len(all_opponent_lineups) < min_lineups:
            print('COULD NOT GENERATE {} OPPONENT LINEUPS'.format(min_lineups), time.ctime())
            continue

        print('GENERATED {} OPPONENT LINEUPS'.format(min_lineups), time.ctime())

        # generate potential lineups for slate
        players = []
        for _, player in slate_board.iterrows():
            player_id = player['PLAYERID']
            name = player['NAME'].split()
            first_name = name[0]
            last_name = name[1] if len(name) > 1 else ''
            positions = player['DFS_POSITION'].split('_')
            team = player['TEAM']
            mu = player[fp_hat_column]
            salary =  player['SALARY']
            std = player[pct_variance_column]*(1 - player[ownership_column])

            player = Player(player_id, first_name, last_name, positions, team, salary, mu, standard_deviation=std/randomness_factor)
            players.append(player)

        optimizer.load_players(players)
        optimizer.set_max_deviation(randomness_factor)
        optimizer.set_min_salary_cap(min_salary)

        total_max_entries = slate_contest_data['MAXENTRIES'].sum()
        n = int(max(2*total_max_entries, 400))
        lineups = optimizer.optimize(n=n, randomness=True)
        
        potential_lineups = pd.DataFrame()
        try:
            for lineup in lineups:
                player_ids = [i.id for i in lineup.lineup]
                real_score = slate_board.loc[slate_board['PLAYERID'].isin(player_ids), 'DKFP'].sum()
                potential_lineups = potential_lineups.append(
                    pd.DataFrame({'PLAYERIDS': [player_ids], 'REALSCORE': [real_score]})
                )
        except:
            print('ERROR: COULD NOT GENERATE LINEUPS')
            continue
            
        print('GENERATED {} POTENTIAL LINEUPS'.format(n), time.ctime())

        # iterate through each contest in the slate
        slate_contest_data = slate_contest_data.loc[
            (slate_contest_data['ENTRYFEE'] <= 250) & (slate_contest_data['ENTRYFEE'] > 1)
        ].sort_values(by='ENTRYFEE', ascending=False)
        for _, contest in slate_contest_data.iterrows():
            contest_id = contest['CONTESTID']
            contest_total_entries = contest['TOTALENTRIES']
            contest_max_entries = contest['MAXENTRIES']
            contest_entry_fee = contest['ENTRYFEE']
            
            result = {} 
            result['DATE'] = date
            result['GAME_COUNT'] = game_count
            result['MAX_ENTRIES'] = contest_max_entries
            result['TOTAL_ENTRIES'] = contest_total_entries
            result['ENTRY_FEE'] = contest_entry_fee
            result['CASHLINE'] = contest['CASHLINE']
            result['TOP_PRIZE'] = contest['TOPPRIZE']
            result['TOP_SCORE'] = contest['TOPSCORE']

            # simulate winnings
            entered_contest_info = contest_info.loc[contest_info['CONTESTID'] == contest_id]
            entered_contest_info['MINPOINTS'] = entered_contest_info['MINPOINTS'].fillna(0)
            rank_to_prize = {}
            prev_i = 0
            prev_prize = 0
            for _, row in entered_contest_info.loc[entered_contest_info['PRIZE'] > 0].iterrows():
                min_rank = int(row['MINRANK'])
                max_rank = int(row['MAXRANK'])
                prize = row['PRIZE']
                for i in range(prev_i+1, min_rank):
                    rank_to_prize[i] = (prize + prev_prize)/2
                for i in range(min_rank, max_rank+1):
                    rank_to_prize[i] = prize
                prev_i = i
                prev_prize = prize

            potential_lineups['WINNINGS'] = 0
            n_samples = int(min(max_n_samples, round(len(all_opponent_lineups)/contest_total_entries)))
            n_simulations_per_sample = int(n_simulations/n_samples)
            for i in range(n_samples):
                opponent_sample = all_opponent_lineups.sample(contest_total_entries)
                for i in range(n_simulations_per_sample):
                    slate_board['SAMPLE'] = slate_board[fp_hat_column]*(1 + normal(0, slate_board[pct_variance_column]))
                    player_performance = slate_board.set_index('PLAYERID')['SAMPLE'].to_dict()

                    opponent_sample['SAMPLE_SCORE'] = opponent_sample['PLAYERIDS'].map(lambda x: -1*sum([player_performance[i] for i in x]))
                    opponent_scores = np.sort(opponent_sample['SAMPLE_SCORE'].to_numpy())[:max_rank]

                    potential_lineups['SAMPLE_SCORE'] = potential_lineups['PLAYERIDS'].map(lambda x: -1*sum([player_performance[i] for i in x]))
                    potential_lineups = potential_lineups.sort_values(by='SAMPLE_SCORE')
                    my_scores = potential_lineups['SAMPLE_SCORE'].to_numpy()

                    prizes = get_prizes(my_scores, opponent_scores, max_rank, rank_to_prize, contest_entry_fee)
                    potential_lineups['WINNINGS'] += prizes

            # enter best lineups
            potential_lineups = potential_lineups.sort_values(by='WINNINGS', ascending=False)
            n_entries = int(min(contest_max_entries, max_entry_fee_per_contest/contest_entry_fee))
            best_lineups = potential_lineups.loc[potential_lineups['WINNINGS'] > 0].head(n_entries)
            n_best_lineups = len(best_lineups)
            potential_lineups = potential_lineups[n_best_lineups:]
            current_pnl = 0
            for _, lineup in best_lineups.iterrows():
                score = lineup['REALSCORE']
                prize = get_contest_prize(entered_contest_info, score)
                
                result['SCORE'] = score
                result['PRIZE'] = prize
                results = results.append(result, ignore_index=True)
                
                contest_pnl = prize - contest_entry_fee
                if not np.isnan(contest_pnl):
                    current_pnl += (prize - contest_entry_fee)

            pnl += current_pnl
            pnl = round(pnl, 2)
            print('{} ({})'.format(str(pnl), str(current_pnl)))

Mon Aug 10 23:23:41 2020
2015-16
2016-17
2017-18
Tue Aug 11 00:01:25 2020
Tue Aug 11 00:01:25 2020 2017-10-18 5ae4d5805689223dfa12d8ad
ERROR: MISSING TEAM DATA
Tue Aug 11 00:01:25 2020 2017-10-18 5ae4dbdb78af173dfeff7c39
ERROR: MISSING TEAM DATA
Tue Aug 11 00:01:25 2020 2017-10-18 5ae4e8b05689223dfa4d3bd7
ERROR: MISSING TEAM DATA
Tue Aug 11 00:01:25 2020 2017-10-19 5ae4d09dc074cf3dff3e3d91
ERROR: MISSING TEAM DATA
Tue Aug 11 00:01:25 2020 2017-10-19 5ae515ee589eaa73373fe802
ERROR: MISSING TEAM DATA
Tue Aug 11 00:01:25 2020 2017-10-20 5ae4cbe2c074cf3dff32fa1b
GENERATED 95365.0 OPPONENT LINEUPS Tue Aug 11 00:03:14 2020
GENERATED 400 POTENTIAL LINEUPS Tue Aug 11 00:03:37 2020
150.0 (150.0)
3545.0 (3395.0)
3884.0 (339.0)
Tue Aug 11 00:04:33 2020 2017-10-20 5ae4d5895689223dfa12e192
GENERATED 118905.0 OPPONENT LINEUPS Tue Aug 11 00:07:10 2020
GENERATED 522 POTENTIAL LINEUPS Tue Aug 11 00:07:42 2020
3434.0 (-450.0)
3134.0 (-300.0)
3034.0 (-100.0)
2884.0 (-150.0)
2844.0 (-40.0)
2817.0 (-27.0)


GENERATED 118905.0 OPPONENT LINEUPS Tue Aug 11 01:32:17 2020
GENERATED 966 POTENTIAL LINEUPS Tue Aug 11 01:33:29 2020
166.74 (-150.0)
16.74 (-150.0)
-358.26 (-375.0)
-314.26 (44.0)
-354.26 (-40.0)
-366.26 (-12.0)
-363.26 (3.0)
-359.26 (4.0)
-670.26 (-311.0)
-559.26 (111.0)
-612.74 (-53.47999999999999)
Tue Aug 11 01:35:31 2020 2017-10-29 5ae51523589eaa73373d82a0
GENERATED 25732.0 OPPONENT LINEUPS Tue Aug 11 01:36:32 2020
GENERATED 400 POTENTIAL LINEUPS Tue Aug 11 01:36:48 2020
-371.74 (241.0)
-522.74 (-151.0)
-1098.88 (-576.14)
Tue Aug 11 01:37:08 2020 2017-10-29 5ae6045cfd67640e26f38b1c
GENERATED 25732.0 OPPONENT LINEUPS Tue Aug 11 01:38:01 2020
GENERATED 400 POTENTIAL LINEUPS Tue Aug 11 01:38:18 2020
141.83 (1240.71)
Tue Aug 11 01:38:30 2020 2017-10-30 5ae4c91ff69c5a3e00128218
GENERATED 118905.0 OPPONENT LINEUPS Tue Aug 11 01:40:41 2020
GENERATED 1210 POTENTIAL LINEUPS Tue Aug 11 01:42:46 2020
-308.17 (-450.0)
-758.17 (-450.0)
-858.17 (-100.0)
-1008.17 (-150.0)
-1158.17 (-150.0)
-1198

GENERATED 118905.0 OPPONENT LINEUPS Tue Aug 11 02:52:24 2020
GENERATED 838 POTENTIAL LINEUPS Tue Aug 11 02:53:24 2020
-5560.25 (-450.0)
-5710.25 (-150.0)
-5810.25 (-100.0)
-5960.25 (-150.0)
-6000.25 (-40.0)
-6670.25 (-670.0)
-7311.25 (-641.0)
-7509.25 (-198.0)
-7536.25 (-27.0)
-7521.25 (15.0)
-7541.25 (-20.0)
-7553.25 (-12.0)
-7681.25 (-128.0)
-8369.75 (-688.5)
-8374.75 (-5.0)
-8380.75 (-6.0)
-8399.75 (-19.0)
-8435.37 (-35.620000000000005)
-8480.37 (-45.0)
-8535.37 (-55.0)
-8537.37 (-2.0)
Tue Aug 11 02:55:31 2020 2017-11-08 5ae4f1bfd69dcc3dfd279719
GENERATED 55145.0 OPPONENT LINEUPS Tue Aug 11 02:56:28 2020
GENERATED 582 POTENTIAL LINEUPS Tue Aug 11 02:56:59 2020
-8761.37 (-224.0)
-8992.8 (-231.43)
-9491.8 (-499.0)
-9545.8 (-54.0)
Tue Aug 11 02:57:44 2020 2017-11-09 5ae4c92e0f94ac3e017aba25
GENERATED 20807.5 OPPONENT LINEUPS Tue Aug 11 02:58:29 2020
GENERATED 400 POTENTIAL LINEUPS Tue Aug 11 02:58:47 2020
-9852.8 (-307.0)
Tue Aug 11 02:58:55 2020 2017-11-09 5ae4da4078af173dfefc1aee
GEN

1988.84 (-204.25)
1302.84 (-686.0)
1255.84 (-47.0)
Tue Aug 11 04:08:35 2020 2017-11-19 5ae4cd6edd71203dfc438848
GENERATED 36762.5 OPPONENT LINEUPS Tue Aug 11 04:09:21 2020
GENERATED 400 POTENTIAL LINEUPS Tue Aug 11 04:09:37 2020
1703.84 (448.0)
1270.84 (-433.0)
Tue Aug 11 04:10:01 2020 2017-11-19 5ae4e628d69dcc3dfd0c50b2
GENERATED 36162.5 OPPONENT LINEUPS Tue Aug 11 04:10:39 2020
GENERATED 400 POTENTIAL LINEUPS Tue Aug 11 04:10:55 2020
1945.84 (675.0)
3018.64 (1072.8)
3051.64 (33.0)
Tue Aug 11 04:11:24 2020 2017-11-19 5ae4e62c5689223dfa43cfa9
GENERATED 80832.5 OPPONENT LINEUPS Tue Aug 11 04:12:43 2020
GENERATED 502 POTENTIAL LINEUPS Tue Aug 11 04:13:08 2020
2901.64 (-150.0)
2506.64 (-395.0)
2494.64 (-12.0)
8150.95 (5656.31)
8130.95 (-20.0)
Tue Aug 11 04:14:20 2020 2017-11-20 5ae4d1c095356d3dfb4bb945
GENERATED 118905.0 OPPONENT LINEUPS Tue Aug 11 04:15:54 2020
GENERATED 400 POTENTIAL LINEUPS Tue Aug 11 04:16:24 2020
8530.95 (400.0)
8430.95 (-100.0)
8330.95 (-100.0)
8180.95 (-150.0)
8140

GENERATED 77205.0 OPPONENT LINEUPS Tue Aug 11 05:55:45 2020
GENERATED 1110 POTENTIAL LINEUPS Tue Aug 11 05:57:19 2020
37512.47 (550.0)
37612.47 (100.0)
37737.47 (125.0)
37049.47 (-688.0)
37009.47 (-40.0)
36759.47 (-250.0)
36732.47 (-27.0)
36727.47 (-5.0)
36765.47 (38.0)
36325.47 (-440.0)
35978.97 (-346.5)
35973.97 (-5.0)
35967.97 (-6.0)
35940.97 (-27.0)
35896.97 (-44.0)
35877.97 (-19.0)
Tue Aug 11 06:00:46 2020 2017-12-02 5ae4cc580f94ac3e0180e043
GENERATED 36762.5 OPPONENT LINEUPS Tue Aug 11 06:02:04 2020
GENERATED 858 POTENTIAL LINEUPS Tue Aug 11 06:02:58 2020
35762.97 (-115.0)
36358.97 (596.0)
36283.97 (-75.0)
36271.97 (-12.0)
35924.26 (-347.71000000000004)
35631.91 (-292.35)
35620.91 (-11.0)
35590.91 (-30.0)
35575.29 (-15.620000000000001)
Tue Aug 11 06:04:12 2020 2017-12-02 5ae4d975c074cf3dff54c67f
GENERATED 45955.0 OPPONENT LINEUPS Tue Aug 11 06:05:17 2020
GENERATED 428 POTENTIAL LINEUPS Tue Aug 11 06:05:35 2020
35490.29 (-85.0)
35322.29 (-168.0)
35015.97 (-306.32)
Tue Aug 11 06:06

36569.44 (5.0)
36537.44 (-32.0)
Tue Aug 11 07:34:26 2020 2017-12-13 5ae4cad0c074cf3dff314d7e
GENERATED 51470.0 OPPONENT LINEUPS Tue Aug 11 07:35:37 2020
GENERATED 444 POTENTIAL LINEUPS Tue Aug 11 07:35:56 2020
36207.44 (-330.0)
36132.44 (-75.0)
35401.44 (-731.0)
35395.44 (-6.0)
Tue Aug 11 07:36:36 2020 2017-12-13 5b834df0dc114c2ea02f5d80
GENERATED 5145.0 OPPONENT LINEUPS Tue Aug 11 07:36:57 2020
GENERATED 400 POTENTIAL LINEUPS Tue Aug 11 07:37:13 2020
35895.44 (500.0)
Tue Aug 11 07:37:18 2020 2017-12-14 5ae4c96c0f94ac3e017b55b2
GENERATED 80260.5 OPPONENT LINEUPS Tue Aug 11 07:38:39 2020
GENERATED 400 POTENTIAL LINEUPS Tue Aug 11 07:38:59 2020
35795.44 (-100.0)
35720.44 (-75.0)
35680.44 (-40.0)
35581.44 (-99.0)
35554.44 (-27.0)
35542.44 (-12.0)
35537.44 (-5.0)
35525.44 (-12.0)
35470.44 (-55.0)
35422.44 (-48.0)
35372.44 (-50.0)
35323.44 (-49.0)
Tue Aug 11 07:41:10 2020 2017-12-14 5ae4f667c074cf3dffaa4910
GENERATED 35277.5 OPPONENT LINEUPS Tue Aug 11 07:42:04 2020
GENERATED 400 POTENTIAL 

56723.5 (-345.31)
56546.57 (-176.93)
56479.05 (-67.52)
Tue Aug 11 09:25:10 2020 2017-12-25 5ae50f6eb819ea73344e5e1b
GENERATED 25732.0 OPPONENT LINEUPS Tue Aug 11 09:26:32 2020
GENERATED 400 POTENTIAL LINEUPS Tue Aug 11 09:26:55 2020
56300.15 (-178.89999999999998)
56250.15 (-50.0)
Tue Aug 11 09:27:15 2020 2017-12-26 5ae4cae7c074cf3dff315b32
GENERATED 118905.0 OPPONENT LINEUPS Tue Aug 11 09:32:04 2020
GENERATED 400 POTENTIAL LINEUPS Tue Aug 11 09:32:46 2020
56300.15 (50.0)
56200.15 (-100.0)
56160.15 (-40.0)
55610.15 (-550.0)
55511.15 (-99.0)
55484.15 (-27.0)
55497.15 (13.0)
55485.15 (-12.0)
55490.15 (5.0)
55478.15 (-12.0)
55440.15 (-38.0)
55404.15 (-36.0)
Tue Aug 11 09:34:40 2020 2017-12-26 5ae5f8ef5641740e23440b16
GENERATED 45955.0 OPPONENT LINEUPS Tue Aug 11 09:36:52 2020
GENERATED 456 POTENTIAL LINEUPS Tue Aug 11 09:37:25 2020
55268.15 (-136.0)
55048.15 (-220.0)
55136.15 (88.0)
55129.15 (-7.0)
Tue Aug 11 09:38:38 2020 2017-12-26 5ae5fde43825540e24ac0677
GENERATED 5145.0 OPPONENT LINEU

Plot overall PNL.

In [ ]:
fig = plt.figure(1, figsize=(30, 15))

# overall PNL
temp = results.copy()
temp['PNL'] = temp['PRIZE'] - temp['ENTRY_FEE']
pnl = temp.groupby(['DATE']).apply(
    lambda x: pd.Series({
        'ENTRY_FEE': x['ENTRY_FEE'].sum(),
        'PRIZE': x['PRIZE'].sum(),
        'PNL': x['PNL'].sum()
    })
).reset_index()
pnl['RUNNING_PNL'] = pnl['PNL'].cumsum()

pnl['COLOR'] = pnl['PNL'].apply(lambda x: 'green' if x > 0 else 'red')
ax1 = sns.barplot(x=pnl.index, y='PNL', data=pnl, palette=pnl['COLOR'])
ax1.set_title('Overall PNL', fontsize=28)

xcoords = {
    pnl.index.min(): ('2017-18 SEASON START', 'k'),
    pnl.index[pnl['DATE'] == '2018-02-08'][0]: ('2017-18 SEASON TRADE DEADLINE', 'm'),
    pnl.index[pnl['DATE'] == '2018-02-22'][0]: ('2017-18 SEASON ALL-STAR RETURN', 'y'),
    pnl.index[pnl['DATE'] == '2018-10-19'][0]: ('2018-19 SEASON START', 'k'),
    pnl.index[pnl['DATE'] == '2019-02-07'][0]: ('2018-19 SEASON TRADE DEADLINE', 'm'),
    pnl.index[pnl['DATE'] == '2019-02-21'][0]: ('2018-19 SEASON ALL-STAR RETURN', 'y'),
    pnl.index[pnl['DATE'] == '2019-10-25'][0]: ('2019-20 SEASON START', 'k'),
    pnl.index[pnl['DATE'] == '2020-02-06'][0]: ('2019-20 SEASON TRADE DEADLINE', 'm'),
    pnl.index[pnl['DATE'] == '2020-02-20'][0]: ('2019-20 SEASON ALL-STAR RETURN', 'y'),
    
}
for xcoord in xcoords:
    ax1.axvline(x=xcoord, label=xcoords[xcoord][0], color=xcoords[xcoord][1], linestyle='dashed')
ax1.legend(bbox_to_anchor=(1, 1.35), loc='upper right', prop={'size': 16})

ax2 = ax1.twinx()
sns.lineplot(x=pnl.index, y='RUNNING_PNL', data=pnl, sort=False, color='tab:blue', ax=ax2)

ax1.set_xlabel('Date', fontsize=22)
ax1.set_ylabel('Daily PNL', fontsize=22)
ax2.set_ylabel('Cumulative PNL', fontsize=22)
ax1.set_xticks([])

ax1.tick_params(axis='both', labelsize=16)
ax2.tick_params(axis='both', labelsize=16)

plt.show()

Break down ROI by contest type, entry fee, and number of games in the slate to determine where my projections generate the most return on investment.


In [ ]:
fig = plt.figure(1, figsize=(40, 30))
gs = fig.add_gridspec(2, 3)

# ROI vs. Entry Fee
temp = results.copy()
temp['ENTRY_BIN'] = pd.cut(temp['ENTRY_FEE'], bins=[0, 5, 10, 25, 50, 100, 250], duplicates='drop')
pnl = temp.groupby(['DATE', 'ENTRY_BIN']).apply(
    lambda x: pd.Series({
        'ENTRY_FEE': x['ENTRY_FEE'].sum(),
        'PRIZE': x['PRIZE'].sum()
    })
).reset_index()
pnl = pnl.merge(boxscores[['DATE', 'SEASON']], on=['DATE'], how='left')
pnl['ROI'] = (pnl['PRIZE'] - pnl['ENTRY_FEE'])/pnl['ENTRY_FEE']

ax4 = fig.add_subplot(gs[1, 0])
sns.barplot(x="ENTRY_BIN", y="ROI", hue="SEASON", data=pnl, ax=ax4)
ax4.set(xlabel="Entry Fee", ylabel="Return on Investment", title="Cash ROI vs. Entry Fee")
ax4.legend()

# ROI vs. Game Count
temp = results.copy()
temp['GAME_BIN'] = pd.cut(temp['GAME_COUNT'], bins=[0, 3, 6, 9, 15], duplicates='drop')
pnl = temp.groupby(['DATE', 'GAME_BIN']).apply(
    lambda x: pd.Series({
        'ENTRY_FEE': x['ENTRY_FEE'].sum(),
        'PRIZE': x['PRIZE'].sum()
    })
).reset_index()
pnl = pnl.merge(boxscores[['DATE', 'SEASON']], on=['DATE'], how='left')
pnl['ROI'] = (pnl['PRIZE'] - pnl['ENTRY_FEE'])/pnl['ENTRY_FEE']

ax5 = fig.add_subplot(gs[1, 1])
sns.barplot(x="GAME_BIN", y="ROI", hue="SEASON", data=pnl, ax=ax5)
ax5.set(xlabel="Number of Games in Slate", ylabel="Return on Investment", title="Cash ROI vs. Game Count")
ax5.legend()

# ROI vs. Entry Fee & Game Count
temp = results.copy()
temp['ENTRY_BIN'] = pd.cut(temp['ENTRY_FEE'], bins=[0, 5, 10, 25, 50, 100, 250], duplicates='drop')
temp['GAME_BIN'] = pd.cut(temp['GAME_COUNT'], bins=[0, 3, 6, 9, 15], duplicates='drop')
pnl = temp.groupby(['DATE', 'ENTRY_BIN', 'GAME_BIN']).apply(
    lambda x: pd.Series({
        'ENTRY_FEE': x['ENTRY_FEE'].sum(),
        'PRIZE': x['PRIZE'].sum()
    })
).reset_index()
pnl = pnl.merge(boxscores[['DATE', 'SEASON']], on=['DATE'], how='left')
pnl['ROI'] = (pnl['PRIZE'] - pnl['ENTRY_FEE'])/pnl['ENTRY_FEE']

ax6 = fig.add_subplot(gs[1, 2])
pnl['UPPER_ENTRY_FEE'] = pnl['ENTRY_BIN'].apply(lambda x: x.right)
pnl['UPPER_GAME_BIN'] = pnl['GAME_BIN'].apply(lambda x: x.right)
scat = pnl.groupby(['UPPER_GAME_BIN', 'UPPER_ENTRY_FEE']).apply(
    lambda x: pd.Series({
        'ROI': x['ROI'].mean()
    })
).reset_index()
sns.scatterplot(x='UPPER_ENTRY_FEE', y="UPPER_GAME_BIN", data=scat, hue='ROI', s=200, palette='coolwarm', ax=ax6)
ax6.set(xlabel="Entry Fee", ylabel="Number of Games in Slate", title="Cash ROI vs. Entry Fee & Game Count")
ax6.legend()

plt.show()

It is important not to overfit by cherry picking the contests in which the projections made money. To try to avoid this, I bucketed the entry fees and slate game counts. I will also only isolate a subset of contests if it makes sense why they were more profitable. The results above show that virtually no subset of cash games are profitable. On the other hand, tournaments whose entry fees are at most $1000 and slate game counts are at most 6 look profitable. This makes sense: higher entry contests attract more intelligent players, and high game slates introduce more variance.

Below I will graph the cumulative PNL of a strategy that only enters contests in the subset described above. I will also include cash games whose entry fees are at most $25 and slate game counts are at most 3 to verify the lack of edge.

In [ ]:
fig, plt.figure(1, figsize=(30, 15))

temp = results.loc[(results['GAME_COUNT'] <= 6)]
temp['PNL'] = temp['PRIZE'] - temp['ENTRY_FEE']
pnl = temp.groupby(['DATE']).apply(
    lambda x: pd.Series({
        'ENTRY_FEE': x['ENTRY_FEE'].sum(),
        'PRIZE': x['PRIZE'].sum(),
        'PNL': x['PNL'].sum()
    })
).reset_index()
pnl['RUNNING_PNL'] = pnl['PNL'].cumsum()

pnl['COLOR'] = pnl['PNL'].apply(lambda x: 'green' if x > 0 else 'red')
ax1 = sns.barplot(x=pnl.index, y='PNL', data=pnl, palette=pnl['COLOR'])
ax1.set_title('Tournament PNL', fontsize=28)

xcoords = {
    pnl.index.min(): ('2017-18 SEASON START', 'k'),
    pnl.index[pnl['DATE'] == '2018-02-08'][0]: ('2017-18 SEASON TRADE DEADLINE', 'm'),
    pnl.index[pnl['DATE'] == '2018-02-22'][0]: ('2017-18 SEASON ALL-STAR RETURN', 'y'),
    pnl.index[pnl['DATE'] == '2018-10-19'][0]: ('2018-19 SEASON START', 'k'),
    pnl.index[pnl['DATE'] == '2019-02-07'][0]: ('2018-19 SEASON TRADE DEADLINE', 'm'),
    pnl.index[pnl['DATE'] == '2019-02-21'][0]: ('2018-19 SEASON ALL-STAR RETURN', 'y'),
    pnl.index[pnl['DATE'] == '2019-10-25'][0]: ('2019-20 SEASON START', 'k'),
    pnl.index[pnl['DATE'] == '2020-02-06'][0]: ('2019-20 SEASON TRADE DEADLINE', 'm'),
    pnl.index[pnl['DATE'] == '2020-02-20'][0]: ('2019-20 SEASON ALL-STAR RETURN', 'y'),
    
}
for xcoord in xcoords:
    ax1.axvline(x=xcoord, color=xcoords[xcoord][1], linestyle='dashed')

ax2 = ax1.twinx()
sns.lineplot(x=pnl.index, y='RUNNING_PNL', data=pnl, sort=False, color='tab:blue', ax=ax2)

ax1.set_xlabel('Date', fontsize=22)
ax1.set_ylabel('Daily PNL', fontsize=22)
ax2.set_ylabel('Cumulative PNL', fontsize=22)
ax1.set_xticks([])

ax1.tick_params(axis='both', labelsize=16)
ax2.tick_params(axis='both', labelsize=16)

plt.show()

I will only focus on tournament contests going forward. The tournament strategy seems like it may actually be profitable. However, there is a lot of variance in PNL, as the cumulative PNL depends entirely on a few days in which it hit big. Most days the strategy loses money, and there are some significant drawdowns.

Below shows the distribution of daily returns of this strategy.

In [ ]:
temp = results.loc[(results['GAME_COUNT'] <= 6)]
temp['PNL'] = temp['PRIZE'] - temp['ENTRY_FEE']
pnl = temp.groupby(['DATE']).apply(
    lambda x: pd.Series({
        'ENTRY_FEE': x['ENTRY_FEE'].sum(),
        'PRIZE': x['PRIZE'].sum(),
        'PNL': x['PNL'].sum()
    })
).reset_index()
pnl['ROI'] = pnl['PNL']/pnl['ENTRY_FEE']
ax = sns.distplot(pnl['ROI'])
ax.set_title('PNL Distribution', fontsize=18)
plt.rcParams["figure.figsize"] = (20, 10)
plt.show()

print('ROI Statistics:')
pnl['ROI'].describe()

In [ ]:
print(pnl['PRIZE'].mean())
print(pnl['ENTRY_FEE'].mean())